In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
import os

root_dir = 'csv_files/Mark Distribution/'

files = [x for x in os.listdir(root_dir) if x[-3:] == 'csv']
dfs = []

for x in files:
    df = pd.read_csv(root_dir + x, index_col=0)
    df['Optimizer'] = x.split('.')[0]
    
    dfs.append(df)
    
df = pd.concat(dfs)

In [3]:
df.head()

,Iterations,Time (sec),Distance,Weighted Mean,Weighted Std,Optimizer
0,150.0,8.138298,0.032432,0.493812,0.105634,cs
1,150.0,5.946815,0.010349,0.496851,0.097481,cs
2,150.0,5.743837,0.016186,0.496569,0.099952,cs
3,150.0,4.618060,0.025684,0.494808,0.103547,cs
4,150.0,4.629465,0.020561,0.495980,0.101718,cs


In [4]:
optimizers = df['Optimizer'].unique()

In [5]:
grouped_distances = df.groupby("Optimizer")["Distance"]
grouped_time = df.groupby("Optimizer")["Time (sec)"]

times = pd.concat([grouped_time.get_group(x) for x in optimizers], axis=1)
distances = pd.concat([grouped_distances.get_group(x) for x in optimizers], axis=1)

distances.columns = optimizers
times.columns = optimizers

In [6]:
labels = distances.columns

min_distances = pd.DataFrame(labels[np.argmin(distances.values, axis=1)]).value_counts()
max_distances = pd.DataFrame(labels[np.argmax(distances.values, axis=1)]).value_counts()

min_distances.name = "Minimum"
max_distances.name = "Maximum"

pd.concat([min_distances, max_distances], axis=1).fillna(0).astype(int)

,Minimum,Maximum
0,,
pfw,25,0
cs,0,13
egd,0,12


In [7]:
labels = times.columns

min_times = pd.DataFrame(labels[np.argmin(times.values, axis=1)]).value_counts()
max_times = pd.DataFrame(labels[np.argmax(times.values, axis=1)]).value_counts()

min_times.name = "Minimum"
max_times.name = "Maximum"

pd.concat([min_times, max_times], axis=1)

,Minimum,Maximum
0,,
pfw,15,5
egd,7,13
cs,3,7


In [23]:
print(r"""\begin{tabular}{l|cc|cc|cc}
\toprule
 & \multicolumn{2}{c}{CS}&\multicolumn{2}{c}{EGD} &\multicolumn{2}{c}{PFW} \\
\midrule
Trial & Distance & Time & Distance & Time & Distance & Time \\
\midrule""")

for i in range(len(distances)):
    distances_vals = distances.iloc[i].values
    times_vals = times.iloc[i].values
    
    min_distance_arg, max_distance_arg = np.argmin(distances_vals), np.argmax(distances_vals)
    min_time_arg, max_time_arg = np.argmin(times_vals), np.argmax(times_vals)
    
    s = f"{i + 1} "
    for index, (distance, time) in enumerate(zip(distances_vals, times_vals)):
        # Distance
        if index == min_distance_arg:
            s += r"& \textbf{" + f"{distance:.6f}" + "} "
        elif index == max_distance_arg:
            s += r"& \underline{" + f"{distance:.6f}" + "} "
        else:
            s += f"& {distance:.6f} "
            
        # Time
        if index == min_time_arg:
            s += r"& \textbf{" + f"{time:.2f}" + "} "
        elif index == max_time_arg:
            s += r"& \underline{" + f"{time:.2f}" + "} "
        else:
            s += f"& {time:.2f} "
            
    s += r"\\"
    print(s)
print(r"\midrule")
print("Average & & " + " & & ".join([f"{x:.2f}" for x in times.mean().values]) + r"\\")

print("\\bottomrule\n\\end{tabular}")

\begin{tabular}{l|cc|cc|cc}
\toprule
 & \multicolumn{2}{c}{CS}&\multicolumn{2}{c}{EGD} &\multicolumn{2}{c}{PFW} \\
\midrule
Trial & Distance & Time & Distance & Time & Distance & Time \\
\midrule
1 & \underline{0.032432} & 8.14 & 0.032426 & \underline{10.98} & \textbf{0.032114} & \textbf{6.02} \\
2 & 0.010349 & \underline{5.95} & \underline{0.010535} & 5.08 & \textbf{0.010101} & \textbf{4.46} \\
3 & 0.016186 & \underline{5.74} & \underline{0.016252} & \textbf{4.93} & \textbf{0.015848} & 5.50 \\
4 & 0.025684 & 4.62 & \underline{0.025726} & \underline{6.19} & \textbf{0.025309} & \textbf{4.51} \\
5 & \underline{0.020561} & 4.63 & 0.020486 & \underline{6.03} & \textbf{0.020213} & \textbf{4.50} \\
6 & \underline{0.016559} & \underline{5.58} & 0.016514 & 5.00 & \textbf{0.016287} & \textbf{4.52} \\
7 & \underline{0.025957} & \underline{5.42} & 0.025867 & \textbf{4.87} & \textbf{0.025757} & 5.38 \\
8 & \underline{0.014506} & 4.77 & 0.014343 & \underline{6.14} & \textbf{0.013504} & \textbf{4.28